In [41]:
import pandas as pd
import requests
import json
import time
import os

In [ ]:
# 1. Find all events and event IDs. An event is something like VCT: Masters - Tokyo 2023

events = []

eventsFilePath = "events/events.json"

with open(eventsFilePath, "r") as f:
    events_data = json.load(f)

events = events_data["pageProps"]["events"]

print("Event IDs and Names:")
print("====================")

eventIds = [event["id"] for event in events]

print("Event IDs:")
print(eventIds)
print(f"\nTotal events found: {len(eventIds)}")

def format_to_csv(df, cols, path):
    df = pd.DataFrame(df)
    df.columns = cols
    df.index = pd.RangeIndex(len(df.index))
    df.to_csv(path)


Event IDs and Names:
Event IDs:
[5574, 5441, 5642, 5639, 5632, 5630, 5624, 5638, 5620, 5609, 5614, 5612, 5619, 5627, 5602, 5586, 5575, 5589, 5588, 5621, 5582, 5581, 5577, 5576, 5564, 5605, 5570, 5542, 5546, 5556, 5559, 5549, 5548, 5591, 5535, 5531, 5585, 5598, 5530, 5461, 5476, 5498, 5583, 5469, 5465, 5440, 5437, 5525, 5523, 5528, 5561, 5496, 5484, 5514, 5472, 5489, 5492, 5438, 5474, 5404, 5517, 5463, 5412, 5527, 5453, 5457, 5443, 5450, 5432, 5481, 5387, 5421, 5407, 5459, 5502, 5406, 5397, 5447, 5390, 5386, 5413, 5428, 5426, 5374, 5377, 5436, 5395, 5351, 5368, 5376, 5371, 5365, 5430, 5362, 5383, 5338, 5375, 5359, 5358, 5320, 5333, 5363, 5328, 5347, 5344, 5336, 5329, 5324, 5303, 5316, 5314, 5341, 5310, 5294, 5306, 5297, 5291, 5263, 5232, 5260, 5228, 5224, 5266, 5230, 5279, 5273, 5221, 5248, 5233, 5256, 5405, 5238, 5281, 5244, 5250, 5251, 5216, 5218, 5214, 5252, 5211, 5213, 5201, 5208, 5189, 5205, 5194, 5191, 5197, 5181, 5167, 5164, 5169, 5162, 5179, 5159, 5185, 5175, 5172, 5148, 5050, 5

In [ ]:


os.makedirs("datasets/consolidated_data", exist_ok=True)

def fetch_consolidated_match_data(event_ids):
    """Fetch and consolidate kill data with economy and location information"""
    
    all_consolidated_kills = []
    
    for event_id in event_ids:
        print(f"Processing event ID: {event_id}")
        
        url = f"https://be-prod.rib.gg/v1/series?completed=true&take=50&eventIds%5B%5D={event_id}"
        series_data = requests.get(url).json()
        match_ids = []
        map_names = {}
        
        for data in series_data.get("data", []):
            for match in data.get("matches", []):
                match_id = match.get("id")
                if match_id:
                    match_ids.append(match_id)
                    if match.get("map") is not None:
                        map_names[match_id] = match["map"].get("name", "Unknown")
                    else:
                        map_names[match_id] = "Unknown"
        
        print(f"Found {len(match_ids)} matches for event {event_id}")
        
        for match_id in match_ids:
            try:
                print(f"  Processing match ID: {match_id} ({map_names[match_id]})")
                
                match_details_url = f"https://be-prod.rib.gg/v1/matches/{match_id}/details"
                match_details = requests.get(match_details_url).json()
                
                map_name = map_names[match_id]
                round_mapping = {}
                player_info = {}
                economy_data = {}
                
                for event in match_details.get("events", []):
                    round_id = event.get("roundId")
                    round_num = event.get("roundNumber")
                    if round_id and round_num is not None:
                        round_mapping[round_num] = round_id
                
                for econ in match_details.get("economies", []):
                    round_id = econ.get("roundId")
                    player_id = econ.get("playerId")
                    agent_id = econ.get("agentId")
                    
                    if player_id and agent_id:
                        if player_id not in player_info:
                            player_info[player_id] = {"agent_id": agent_id}
                    
                    if round_id and player_id:
                        key = f"{round_id}_{player_id}"
                        economy_data[key] = econ
                
                kill_events = [e for e in match_details.get("events", []) if e.get("eventType") == "kill"]
                
                for kill_event in kill_events:
                    round_id = kill_event.get("roundId")
                    round_number = kill_event.get("roundNumber")
                    round_time = kill_event.get("roundTimeMillis")
                    kill_id = kill_event.get("killId")
                    killer_id = kill_event.get("playerId")
                    victim_id = kill_event.get("referencePlayerId")
                    weapon_id = kill_event.get("weaponId")
                    
                    if not (round_id and killer_id and victim_id):
                        continue
                    
                    # Find killer economy data
                    killer_economy = economy_data.get(f"{round_id}_{killer_id}", {})
                    # Find victim economy data  
                    victim_economy = economy_data.get(f"{round_id}_{victim_id}", {})
                    
                    # Find locations near the kill time
                    killer_locations = []
                    victim_locations = []
                    
                    for loc in match_details.get("locations", []):
                        if loc.get("roundNumber") != round_number:
                            continue
                            
                        time_diff = abs(loc.get("roundTimeMillis", 0) - round_time)
                        
                        # Use a threshold of 2 seconds (2000ms) for location proximity to kill
                        if time_diff <= 2000:
                            if loc.get("playerId") == killer_id:
                                killer_locations.append((time_diff, loc))
                            elif loc.get("playerId") == victim_id:
                                victim_locations.append((time_diff, loc))
                    
                    killer_location = min(killer_locations, key=lambda x: x[0])[1] if killer_locations else {}
                    victim_location = min(victim_locations, key=lambda x: x[0])[1] if victim_locations else {}
                    
                    plant_events = [e for e in match_details.get("events", []) 
                                   if e.get("roundId") == round_id and e.get("eventType") == "plant"]
                    plant_time = plant_events[0].get("roundTimeMillis") if plant_events else None
                    plant_site = None
                    
                    consolidated_kill = {
                        # Basic match/round info
                        "map_name": map_name,
                        "match_id": match_id,
                        "round_id": round_id,
                        "round_number": round_number,
                        "time": round_time,
                        "plant_time": plant_time,
                        "plant_site": plant_site,
                        
                        # Kill info
                        "kill_id": kill_id,
                        "weapon_id": weapon_id,
                        "first_kill": kill_event.get("firstKill", False),
                        "traded_by_kill_id": kill_event.get("tradedByKillId"),
                        "traded_for_kill_id": kill_event.get("tradedForKillId"),
                        
                        # Killer info
                        "killer_id": killer_id,
                        "killer_agent_id": killer_economy.get("agentId"),
                        "killer_weapon_id": killer_economy.get("weaponId"),
                        "killer_armor_id": killer_economy.get("armorId"),
                        "killer_loadout_value": killer_economy.get("loadoutValue"),
                        "killer_x": killer_location.get("locationX"),
                        "killer_y": killer_location.get("locationY"),
                        "killer_view_radians": killer_location.get("viewRadians"),
                        "side": "atk" if killer_economy.get("attackingSide") else "def",
                        
                        # Victim info
                        "victim_id": victim_id,
                        "victim_agent_id": victim_economy.get("agentId"),
                        "victim_weapon_id": victim_economy.get("weaponId"),
                        "victim_armor_id": victim_economy.get("armorId"),
                        "victim_loadout_value": victim_economy.get("loadoutValue"),
                        "victim_x": victim_location.get("locationX"),
                        "victim_y": victim_location.get("locationY"),
                        "victim_view_radians": victim_location.get("viewRadians"),
                        
                        "event_id": event_id
                    }
                    
                    all_consolidated_kills.append(consolidated_kill)
                
                print(f"    Added {len(kill_events)} kill events from match")
                time.sleep(0.5)  # Be nice to the API
                
            except Exception as e:
                print(f"  Error processing match {match_id}: {str(e)}")
    
    return all_consolidated_kills


In [46]:
if __name__ == "__main__":
    with open("events/events.json", "r") as f:
        events_data = json.load(f)
    
    event_ids = [event["id"] for event in events_data["pageProps"]["events"]]
    print(f"Processing {len(event_ids)} events")
    
    # Fetch consolidated data
    consolidated_kills = fetch_consolidated_match_data(event_ids)
    
    # Create DataFrame and save to CSV
    if consolidated_kills:
        df = pd.DataFrame(consolidated_kills)
        
        # Save all data
        df.to_csv("datasets/consolidated_data/all_kill_data_consolidated.csv", index=False)
        print(f"Saved {len(df)} consolidated kill records")
        
        # Also save by map for your models
        for map_name, map_group in df.groupby("map_name"):
            map_filename = f"datasets/consolidated_data/{map_name.lower()}_kill_data_consolidated.csv"
            map_group.to_csv(map_filename, index=False)
            print(f"Saved {len(map_group)} kill records for {map_name}")
    else:
        print("No data collected")

Processing 200 events
Processing event ID: 5574
Found 0 matches for event 5574
Processing event ID: 5441
Found 0 matches for event 5441
Processing event ID: 5642
Found 18 matches for event 5642
  Processing match ID: 203244 (Unknown)
    Added 0 kill events from match
  Processing match ID: 203245 (Unknown)
    Added 0 kill events from match
  Processing match ID: 203246 (Unknown)
    Added 0 kill events from match
  Processing match ID: 203235 (Haven)
    Added 0 kill events from match
  Processing match ID: 203236 (Pearl)
    Added 0 kill events from match
  Processing match ID: 203237 (Unknown)
    Added 0 kill events from match
  Processing match ID: 203232 (Sunset)
    Added 0 kill events from match
  Processing match ID: 203233 (Split)
    Added 0 kill events from match
  Processing match ID: 203234 (Unknown)
    Added 0 kill events from match
  Processing match ID: 203229 (Haven)
    Added 0 kill events from match
  Processing match ID: 203230 (Icebox)
    Added 0 kill events fr

KeyboardInterrupt: 